In [1]:
!ls  ..

DeepSpeech-2.x	asr  docker.md	ftp.py	proxy.sh  tfe  tts  tutorial


In [2]:
espnet_dir = '../asr/espnet/'
ds2_dir = '../DeepSpeech-2.x/'

In [3]:
!ls $espnet_dir

CONTRIBUTING.md  ci	 egs	 espnet.egg-info  setup.py    tools
LICENSE		 doc	 egs2	 espnet2	  test	      utils
README.md	 docker  espnet  setup.cfg	  test_utils


In [4]:
!ls $ds2_dir

LICENSE       deepspeech  examples	    speechnn	 tmp
README.md     doc	  requirements.txt  tests	 tools
README_cn.md  env.sh	  setup.sh	    third_party  utils


In [5]:
import configargparse
import logging
import os
import random
import sys
import numpy as np


In [6]:
#load utt

In [7]:
!ls $espnet_dir/egs/librispeech/asr1/dump/dev_clean/deltafalse/data_unigram5000.json

../asr/espnet//egs/librispeech/asr1/dump/dev_clean/deltafalse/data_unigram5000.json


In [8]:
!ls $espnet_dir/egs/librispeech/asr1/asr1/exp/train_960_pytorch_train_specaug/results/snapshot.ep.27

../asr/espnet//egs/librispeech/asr1/asr1/exp/train_960_pytorch_train_specaug/results/snapshot.ep.27


In [9]:
import json
import logging
import torch
from espnet.asr.asr_utils import add_results_to_json
from espnet.asr.asr_utils import get_model_conf
from espnet.asr.asr_utils import torch_load
from espnet.asr.pytorch_backend.asr import load_trained_model
from espnet.nets.asr_interface import ASRInterface
from espnet.nets.batch_beam_search import BatchBeamSearch
from espnet.nets.beam_search import BeamSearch
from espnet.nets.lm_interface import dynamic_import_lm
from espnet.nets.scorer_interface import BatchScorerInterface
from espnet.nets.scorers.length_bonus import LengthBonus
from espnet.utils.deterministic_utils import set_deterministic_pytorch
from espnet.utils.io_utils import LoadInputsAndTargets

In [10]:
recog_json="../asr/espnet//egs/librispeech/asr1/dump/dev_clean/deltafalse/data_unigram5000.json"
# read json data
with open(recog_json, "rb") as f:
    js = json.load(f)["utts"]

In [11]:
for item in js.items():
    print(item)
    break;

('1272-128104-0000', {'input': [{'feat': '/workspace/zhanghui/asr/espnet/egs/librispeech/asr1/dump/dev_clean/deltafalse/feats.1.ark:17', 'name': 'input1', 'shape': [584, 83]}], 'output': [{'name': 'target1', 'shape': [22, 5002], 'text': 'MISTER QUILTER IS THE APOSTLE OF THE MIDDLE CLASSES AND WE ARE GLAD TO WELCOME HIS GOSPEL', 'token': '▁MISTER ▁QUI L TER ▁IS ▁THE ▁A PO ST LE ▁OF ▁THE ▁MIDDLE ▁CLASSES ▁AND ▁WE ▁ARE ▁GLAD ▁TO ▁WELCOME ▁HIS ▁GOSPEL', 'tokenid': '3008 3630 245 410 2598 4502 482 352 399 252 3204 4502 2974 1165 627 4845 689 2209 4577 4859 2391 2237'}], 'utt2spk': '1272-128104'})


In [12]:
ckpt="../asr/espnet//egs/librispeech/asr1/asr1/exp/train_960_pytorch_train_specaug/results/snapshot.ep.27"
model_state_dict = torch.load(ckpt, map_location=lambda storage, loc: storage)["model"]
for key, val in model_state_dict.items():
    print(key, "\t", val.shape, "\t", val.dtype)

encoder.embed.conv.0.weight 	 torch.Size([512, 1, 3, 3]) 	 torch.float32
encoder.embed.conv.0.bias 	 torch.Size([512]) 	 torch.float32
encoder.embed.conv.2.weight 	 torch.Size([512, 512, 3, 3]) 	 torch.float32
encoder.embed.conv.2.bias 	 torch.Size([512]) 	 torch.float32
encoder.embed.out.0.weight 	 torch.Size([512, 10240]) 	 torch.float32
encoder.embed.out.0.bias 	 torch.Size([512]) 	 torch.float32
encoder.encoders.0.self_attn.linear_q.weight 	 torch.Size([512, 512]) 	 torch.float32
encoder.encoders.0.self_attn.linear_q.bias 	 torch.Size([512]) 	 torch.float32
encoder.encoders.0.self_attn.linear_k.weight 	 torch.Size([512, 512]) 	 torch.float32
encoder.encoders.0.self_attn.linear_k.bias 	 torch.Size([512]) 	 torch.float32
encoder.encoders.0.self_attn.linear_v.weight 	 torch.Size([512, 512]) 	 torch.float32
encoder.encoders.0.self_attn.linear_v.bias 	 torch.Size([512]) 	 torch.float32
encoder.encoders.0.self_attn.linear_out.weight 	 torch.Size([512, 512]) 	 torch.float32
encoder.encode

In [13]:
for key, val in model_state_dict.items():
    if 'decoders' in key or 'decoder' in key:
        continue
    print(key, "\t", val.shape, "\t", val.dtype)

encoder.embed.conv.0.weight 	 torch.Size([512, 1, 3, 3]) 	 torch.float32
encoder.embed.conv.0.bias 	 torch.Size([512]) 	 torch.float32
encoder.embed.conv.2.weight 	 torch.Size([512, 512, 3, 3]) 	 torch.float32
encoder.embed.conv.2.bias 	 torch.Size([512]) 	 torch.float32
encoder.embed.out.0.weight 	 torch.Size([512, 10240]) 	 torch.float32
encoder.embed.out.0.bias 	 torch.Size([512]) 	 torch.float32
encoder.encoders.0.self_attn.linear_q.weight 	 torch.Size([512, 512]) 	 torch.float32
encoder.encoders.0.self_attn.linear_q.bias 	 torch.Size([512]) 	 torch.float32
encoder.encoders.0.self_attn.linear_k.weight 	 torch.Size([512, 512]) 	 torch.float32
encoder.encoders.0.self_attn.linear_k.bias 	 torch.Size([512]) 	 torch.float32
encoder.encoders.0.self_attn.linear_v.weight 	 torch.Size([512, 512]) 	 torch.float32
encoder.encoders.0.self_attn.linear_v.bias 	 torch.Size([512]) 	 torch.float32
encoder.encoders.0.self_attn.linear_out.weight 	 torch.Size([512, 512]) 	 torch.float32
encoder.encode

encoder.encoders.9.norm1.bias 	 torch.Size([512]) 	 torch.float32
encoder.encoders.9.norm2.weight 	 torch.Size([512]) 	 torch.float32
encoder.encoders.9.norm2.bias 	 torch.Size([512]) 	 torch.float32
encoder.encoders.10.self_attn.linear_q.weight 	 torch.Size([512, 512]) 	 torch.float32
encoder.encoders.10.self_attn.linear_q.bias 	 torch.Size([512]) 	 torch.float32
encoder.encoders.10.self_attn.linear_k.weight 	 torch.Size([512, 512]) 	 torch.float32
encoder.encoders.10.self_attn.linear_k.bias 	 torch.Size([512]) 	 torch.float32
encoder.encoders.10.self_attn.linear_v.weight 	 torch.Size([512, 512]) 	 torch.float32
encoder.encoders.10.self_attn.linear_v.bias 	 torch.Size([512]) 	 torch.float32
encoder.encoders.10.self_attn.linear_out.weight 	 torch.Size([512, 512]) 	 torch.float32
encoder.encoders.10.self_attn.linear_out.bias 	 torch.Size([512]) 	 torch.float32
encoder.encoders.10.feed_forward.w_1.weight 	 torch.Size([2048, 512]) 	 torch.float32
encoder.encoders.10.feed_forward.w_1.bias 

In [157]:
import paddle
# dump torch model encoder to paddle encoder

def torch_paddle_param(model_state_dict):
    paddle_state_dict = {}
    for n, p in model_state_dict.items():
        print(f'-> {n}    {p.ndim}')
        
        # change norm.mean and norm variance
        name_change=True
        if 'norm.running_mean' in n:
            new_n = n.replace('norm.running_', 'norm._')
        elif 'norm.running_var' in n:
            new_n = n.replace('norm.running_var', 'norm._variance')
        else:
            name_change=False
            new_n = n
        if name_change:
            print(f"{n} -> {new_n}")

        p = p.cpu().detach().numpy()
        
        # weight with is rank 2, transpose it
        if n.endswith('weight') and p.ndim == 2:
            new_p = p.T
            print(f"\t{n}: {p.shape} -> {new_p.shape}")
        else:
            new_p = p
        
        # embedding layer
        if 'decoder.embed.0.weight' in n:
            new_p = p

        if 'global_cmvn.mean' in n:
            print("cmvn:", p, p.dtype)

        paddle_state_dict[new_n] = new_p
    return paddle_state_dict

In [158]:
paddle_model_state_dict = torch_paddle_param(model_state_dict)

-> encoder.embed.conv.0.weight    4
-> encoder.embed.conv.0.bias    1
-> encoder.embed.conv.2.weight    4
-> encoder.embed.conv.2.bias    1
-> encoder.embed.out.0.weight    2
	encoder.embed.out.0.weight: (512, 10240) -> (10240, 512)
-> encoder.embed.out.0.bias    1
-> encoder.encoders.0.self_attn.linear_q.weight    2
	encoder.encoders.0.self_attn.linear_q.weight: (512, 512) -> (512, 512)
-> encoder.encoders.0.self_attn.linear_q.bias    1
-> encoder.encoders.0.self_attn.linear_k.weight    2
	encoder.encoders.0.self_attn.linear_k.weight: (512, 512) -> (512, 512)
-> encoder.encoders.0.self_attn.linear_k.bias    1
-> encoder.encoders.0.self_attn.linear_v.weight    2
	encoder.encoders.0.self_attn.linear_v.weight: (512, 512) -> (512, 512)
-> encoder.encoders.0.self_attn.linear_v.bias    1
-> encoder.encoders.0.self_attn.linear_out.weight    2
	encoder.encoders.0.self_attn.linear_out.weight: (512, 512) -> (512, 512)
-> encoder.encoders.0.self_attn.linear_out.bias    1
-> encoder.encoders.0.fe

In [159]:
for key, val in paddle_model_state_dict.items():
    print(key, "\t", val.shape, "\t", val.dtype)

encoder.embed.conv.0.weight 	 (512, 1, 3, 3) 	 float32
encoder.embed.conv.0.bias 	 (512,) 	 float32
encoder.embed.conv.2.weight 	 (512, 512, 3, 3) 	 float32
encoder.embed.conv.2.bias 	 (512,) 	 float32
encoder.embed.out.0.weight 	 (10240, 512) 	 float32
encoder.embed.out.0.bias 	 (512,) 	 float32
encoder.encoders.0.self_attn.linear_q.weight 	 (512, 512) 	 float32
encoder.encoders.0.self_attn.linear_q.bias 	 (512,) 	 float32
encoder.encoders.0.self_attn.linear_k.weight 	 (512, 512) 	 float32
encoder.encoders.0.self_attn.linear_k.bias 	 (512,) 	 float32
encoder.encoders.0.self_attn.linear_v.weight 	 (512, 512) 	 float32
encoder.encoders.0.self_attn.linear_v.bias 	 (512,) 	 float32
encoder.encoders.0.self_attn.linear_out.weight 	 (512, 512) 	 float32
encoder.encoders.0.self_attn.linear_out.bias 	 (512,) 	 float32
encoder.encoders.0.feed_forward.w_1.weight 	 (512, 2048) 	 float32
encoder.encoders.0.feed_forward.w_1.bias 	 (2048,) 	 float32
encoder.encoders.0.feed_forward.w_2.weight 	 (2048

In [161]:
for key, val in model_state_dict.items():
    if 'decoder.embed.0.weight' in key:
        val = val
    elif key.endswith('weight') and val.ndim == 2:
        val = val.T  
    print(np.allclose(val, paddle_model_state_dict[key]))

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [47]:
load_inputs_and_targets = LoadInputsAndTargets(
        mode="asr",
        load_input=True,
        load_output=True,
        sort_in_input_length=False,
        preprocess_conf=None,
        preprocess_args={"train": False},
    )

In [48]:
with torch.no_grad():
        idx = 0
        name = list(js.keys())[idx]
        logging.info("(%d/%d) decoding " + name, idx, len(js.keys()))
        batch = [(name, js[name])]
        print(batch, '\n')
        bdata = load_inputs_and_targets(batch)
        print(len(bdata))
        feat = bdata[0][0]
        label = bdata[1][0]
        print(feat)
        print(label)

[('1272-128104-0000', {'input': [{'feat': '/workspace/zhanghui/asr/espnet/egs/librispeech/asr1/dump/dev_clean/deltafalse/feats.1.ark:17', 'name': 'input1', 'shape': [584, 83]}], 'output': [{'name': 'target1', 'shape': [22, 5002], 'text': 'MISTER QUILTER IS THE APOSTLE OF THE MIDDLE CLASSES AND WE ARE GLAD TO WELCOME HIS GOSPEL', 'token': '▁MISTER ▁QUI L TER ▁IS ▁THE ▁A PO ST LE ▁OF ▁THE ▁MIDDLE ▁CLASSES ▁AND ▁WE ▁ARE ▁GLAD ▁TO ▁WELCOME ▁HIS ▁GOSPEL', 'tokenid': '3008 3630 245 410 2598 4502 482 352 399 252 3204 4502 2974 1165 627 4845 689 2209 4577 4859 2391 2237'}], 'utt2spk': '1272-128104'})] 

2
[[-0.74641937 -0.58889526 -0.79081845 ...  1.2798967   0.22447938
  -0.11255249]
 [-0.74641937 -0.574557   -0.9586555  ...  1.3187659   0.24862868
   0.06486106]
 [-0.51503205 -0.41683638 -0.6949115  ...  1.3654089   0.28082776
   0.04357147]
 ...
 [-0.65000796 -0.54588056 -0.9586555  ...  0.7260699  -0.24240735
  -0.19061443]
 [-0.74641937 -0.6175717  -0.8387718  ...  0.7131443  -0.2343576
 

In [49]:
!ls $espnet_dir/egs/librispeech/asr1/asr1/exp/train_960_pytorch_train_specaug/results

acc.png		model.loss.best  snapshot.ep.16  snapshot.ep.23  snapshot.ep.6
att_ws		snapshot.ep.1	 snapshot.ep.17  snapshot.ep.24  snapshot.ep.7
ctc_prob	snapshot.ep.10	 snapshot.ep.18  snapshot.ep.25  snapshot.ep.8
log		snapshot.ep.11	 snapshot.ep.19  snapshot.ep.26  snapshot.ep.9
loss.png	snapshot.ep.12	 snapshot.ep.2	 snapshot.ep.27
model.acc.best	snapshot.ep.13	 snapshot.ep.20  snapshot.ep.3
model.json	snapshot.ep.14	 snapshot.ep.21  snapshot.ep.4
model.json.bak	snapshot.ep.15	 snapshot.ep.22  snapshot.ep.5


In [20]:
model, train_args = load_trained_model(ckpt)
model.eval()
print(model)
print(train_args)

/workspace/zhanghui/asr/espnet/tools/anaconda/envs/espnet/lib/python3.8/site-packages/matplotlib/backend_bases.py:57: DeprecationWarning: PILLOW_VERSION is deprecated and will be removed in Pillow 9 (2022-01-02). Use __version__ instead.
  from PIL import PILLOW_VERSION


E2E(
  (encoder): Encoder(
    (embed): Conv2dSubsampling(
      (conv): Sequential(
        (0): Conv2d(1, 512, kernel_size=(3, 3), stride=(2, 2))
        (1): ReLU()
        (2): Conv2d(512, 512, kernel_size=(3, 3), stride=(2, 2))
        (3): ReLU()
      )
      (out): Sequential(
        (0): Linear(in_features=10240, out_features=512, bias=True)
        (1): PositionalEncoding(
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (encoders): MultiSequential(
      (0): EncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linear_q): Linear(in_features=512, out_features=512, bias=True)
          (linear_k): Linear(in_features=512, out_features=512, bias=True)
          (linear_v): Linear(in_features=512, out_features=512, bias=True)
          (linear_out): Linear(in_features=512, out_features=512, bias=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
        (feed_forward): PositionwiseFeedForward(
          (w_1): Linear(i

In [21]:
device='cpu'
with torch.no_grad():
    enc = model.encode(torch.as_tensor(feat).to(device=device))
    print(enc.numpy())

[[ 0.0084809  -0.03854299  0.05893369 ...  0.02430487 -0.04398455
  -0.05841548]
 [ 0.00554978 -0.03724072  0.05680261 ...  0.02338517 -0.03312957
  -0.05391716]
 [ 0.00168099 -0.04298185  0.06165742 ...  0.02442379 -0.0287394
  -0.05569264]
 ...
 [ 0.23188232  0.0513467  -0.02806944 ...  0.02038751  0.00450835
  -0.207303  ]
 [ 0.3011615   0.0425128  -0.00147884 ... -0.03319022 -0.01043977
  -0.20581883]
 [ 0.32673094  0.0756444  -0.04818322 ... -0.04932987  0.02738191
  -0.23118067]]


/workspace/zhanghui/asr/espnet/tools/anaconda/envs/espnet/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
# def recog_v2(args):
#     """Decode with custom models that implements ScorerInterface.

#     Notes:
#         The previous backend espnet.asr.pytorch_backend.asr.recog
#         only supports E2E and RNNLM

#     Args:
#         args (namespace): The program arguments.
#         See py:func:`espnet.bin.asr_recog.get_parser` for details

#     """
#     set_deterministic_pytorch(args)
#     model, train_args = load_trained_model(args.model)

#     model.eval()

#     if args.ngpu == 1:
#         device = "cuda"
#     else:
#         device = "cpu"
#     dtype = getattr(torch, args.dtype)

#     model.to(device=device, dtype=dtype).eval()
#     with torch.no_grad():
#         for idx, name in enumerate(js.keys(), 1):
#             logging.info("(%d/%d) decoding " + name, idx, len(js.keys()))
#             batch = [(name, js[name])]
#             feat = load_inputs_and_targets(batch)[0][0]
#             enc = model.encode(torch.as_tensor(feat).to(device=device, dtype=dtype))
#             nbest_hyps = beam_search(
#                 x=enc, maxlenratio=args.maxlenratio, minlenratio=args.minlenratio
#             )
#             nbest_hyps = [
#                 h.asdict() for h in nbest_hyps[: min(len(nbest_hyps), args.nbest)]
#             ]
#             new_js[name] = add_results_to_json(
#                 js[name], nbest_hyps, train_args.char_list
#             )


# def torch_load(path, model):
#     """Load torch model states.

#     Args:
#         path (str): Model path or snapshot file path to be loaded.
#         model (torch.nn.Module): Torch model.

#     """
#     if "snapshot" in os.path.basename(path):
#         model_state_dict = torch.load(path, map_location=lambda storage, loc: storage)[
#             "model"
#         ]
#     else:
#         model_state_dict = torch.load(path, map_location=lambda storage, loc: storage)

#     if hasattr(model, "module"):
#         model.module.load_state_dict(model_state_dict)
#     else:
#         model.load_state_dict(model_state_dict)

#     del model_state_dict

In [23]:
x = np.expand_dims(feat, 0)
ilen = np.array(feat.shape[1]).reshape(1)
y = np.expand_dims(label, 0)
print(x.shape)
print(ilen.shape)
print(y.shape)
loss = model(
    torch.as_tensor(x).to(device=device), 
    torch.as_tensor(ilen).to(device),
    torch.as_tensor(y).to(device))
print(loss)
print(loss.grad)

(1, 584, 83)
(1,)
(1, 22)
tensor(115.3222, grad_fn=<AddBackward0>)
None


<ipython-input-23-81ae1e6efdc8>:12: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  print(loss.grad)


In [24]:
# encoder + ctc  grad

In [25]:
!ls ../asr/espnet//egs/librispeech/asr1/asr1/exp/train_960_pytorch_train_specaug/results/

acc.png		model.loss.best  snapshot.ep.16  snapshot.ep.23  snapshot.ep.6
att_ws		snapshot.ep.1	 snapshot.ep.17  snapshot.ep.24  snapshot.ep.7
ctc_prob	snapshot.ep.10	 snapshot.ep.18  snapshot.ep.25  snapshot.ep.8
log		snapshot.ep.11	 snapshot.ep.19  snapshot.ep.26  snapshot.ep.9
loss.png	snapshot.ep.12	 snapshot.ep.2	 snapshot.ep.27
model.acc.best	snapshot.ep.13	 snapshot.ep.20  snapshot.ep.3
model.json	snapshot.ep.14	 snapshot.ep.21  snapshot.ep.4
model.json.bak	snapshot.ep.15	 snapshot.ep.22  snapshot.ep.5


In [26]:

def make_pad_mask(lengths, xs=None, length_dim=-1):
    if length_dim == 0:
        raise ValueError("length_dim cannot be 0: {}".format(length_dim))

    if not isinstance(lengths, list):
        lengths = lengths.tolist()
    bs = int(len(lengths))
    if xs is None:
        maxlen = int(max(lengths))
    else:
        maxlen = xs.size(length_dim)

    seq_range = torch.arange(0, maxlen, dtype=torch.int64)
    seq_range_expand = seq_range.unsqueeze(0).expand(bs, maxlen)
    seq_length_expand = seq_range_expand.new(lengths).unsqueeze(-1)
    mask = seq_range_expand >= seq_length_expand

    if xs is not None:
        assert xs.size(0) == bs, (xs.size(0), bs)

        if length_dim < 0:
            length_dim = xs.dim() + length_dim
        # ind = (:, None, ..., None, :, , None, ..., None)
        ind = tuple(
            slice(None) if i in (0, length_dim) else None for i in range(xs.dim())
        )
        mask = mask[ind].expand_as(xs).to(xs.device)
    return mask


def make_non_pad_mask(lengths, xs=None, length_dim=-1):
    return ~make_pad_mask(lengths, xs, length_dim)

In [57]:
with torch.no_grad():
        idx = 1
        name = list(js.keys())[idx]
        logging.info("(%d/%d) decoding " + name, idx, len(js.keys()))
        batch = [(name, js[name])]
        print(batch, '\n')
        bdata = load_inputs_and_targets(batch)
        print(len(bdata))
        feat2 = bdata[0][0]
        label2 = bdata[1][0]
        print(feat2)
        print(label2)

[('1272-128104-0001', {'input': [{'feat': '/workspace/zhanghui/asr/espnet/egs/librispeech/asr1/dump/dev_clean/deltafalse/feats.1.ark:49191', 'name': 'input1', 'shape': [480, 83]}], 'output': [{'name': 'target1', 'shape': [14, 5002], 'text': "NOR IS MISTER QUILTER'S MANNER LESS INTERESTING THAN HIS MATTER", 'token': "▁NOR ▁IS ▁MISTER ▁QUI L TER ' S ▁MANNER ▁LESS ▁INTERESTING ▁THAN ▁HIS ▁MATTER", 'tokenid': '3152 2598 3008 3630 245 410 2 389 2893 2770 2572 4499 2391 2926'}], 'utt2spk': '1272-128104'})] 

2
[[-0.5260203  -0.440242   -0.8399645  ...  1.4257575  -0.46800017
  -0.44417572]
 [-0.40732563 -0.37577838 -0.72303843 ...  1.4127053  -0.5211444
  -0.33839935]
 [-0.57349813 -0.40801018 -0.69965315 ...  1.4518621  -0.5683837
   0.13844776]
 ...
 [-0.8108874  -0.6497487  -0.9568907  ...  1.2756567  -1.032064
  -0.38541245]
 [-0.71593165 -0.5047056  -0.7464236  ...  1.3735485  -1.0094705
   0.1451639 ]
 [-0.47854233 -0.440242   -0.9335055  ...  1.3996531  -0.96428335
  -0.38541245]]
[31

In [58]:
model, train_args = load_trained_model(ckpt)
model.eval()

x = np.expand_dims(feat, 0)
ilen = np.array(feat.shape[0]).reshape(1)
y = np.expand_dims(label, 0)
print(x.shape)
print(ilen)
print(ilen.shape)
print(y.shape)
print(y)

device='cpu'
model.to(device)
model.zero_grad()
print(model.adim)

xs_pad = torch.as_tensor(x).to(device=device)
ilens = torch.as_tensor(ilen).to(device)
ys_pad = torch.as_tensor(y).to(device)
    

xs_pad = xs_pad[:, : max(ilens)]
src_mask = make_non_pad_mask(ilens.tolist()).to(xs_pad.device).unsqueeze(-2)
hs_pad, hs_mask = model.encoder(xs_pad, src_mask)
print(hs_pad.sum())
batch_size = xs_pad.size(0)
hs_len = hs_mask.view(batch_size, -1).sum(1)
loss = model.ctc(hs_pad.view(batch_size, -1, model.adim), hs_len, ys_pad)

print('loss', loss)
loss.retain_grad()
loss.backward()
print(loss.grad)

(1, 584, 83)
[584]
(1,)
(1, 22)
[[3008 3630  245  410 2598 4502  482  352  399  252 3204 4502 2974 1165
   627 4845  689 2209 4577 4859 2391 2237]]
512
tensor(122.5361, grad_fn=<SumBackward0>)
loss tensor(5.1865, grad_fn=<SumBackward0>)
tensor(1.)


In [59]:
for key, val in model.named_parameters():
    print(key, "\t",  val.grad.detach().numpy())

encoder.embed.conv.0.weight 	 [[[[  0.         0.         0.      ]
   [  0.         0.         0.      ]
   [  0.         0.         0.      ]]]


 [[[ -8.080228  -9.529402  -9.047782]
   [-10.095183 -11.103379 -11.473026]
   [-10.447368 -10.538891 -12.96816 ]]]


 [[[  0.         0.         0.      ]
   [  0.         0.         0.      ]
   [  0.         0.         0.      ]]]


 ...


 [[[  0.         0.         0.      ]
   [  0.         0.         0.      ]
   [  0.         0.         0.      ]]]


 [[[  0.         0.         0.      ]
   [  0.         0.         0.      ]
   [  0.         0.         0.      ]]]


 [[[  0.         0.         0.      ]
   [  0.         0.         0.      ]
   [  0.         0.         0.      ]]]]
encoder.embed.conv.0.bias 	 [ 0.0000000e+00 -7.0013804e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
  3.8619192e+00  0.0000000e+00  0.0000000e+00

encoder.encoders.4.norm2.weight 	 [-3.78467254e-02 -5.01782028e-03  4.35798280e-02 -4.29600067e-02
 -1.67928115e-02 -2.13645063e-02  2.03470483e-01  2.55802390e-03
 -4.52783816e-02  5.46755977e-02 -3.54202464e-02  6.08153734e-03
  1.35528333e-02  3.66946496e-03 -4.35056165e-02 -6.39525801e-02
 -1.02985382e-01  9.16976571e-01 -2.18466111e-03  2.48014610e-02
  4.00162376e-02 -2.99065262e-02 -1.90419471e-03  6.52313530e-02
 -5.74541651e-02  7.96481501e-03  2.17269827e-02 -1.05732679e-02
 -1.10418260e-01 -5.39662614e-02  4.24661189e-02 -1.54174110e-02
 -1.52716835e-04 -3.12010776e-02  1.21810162e+00 -5.17224148e-02
 -4.35296632e-02  1.38233444e-02  1.97195355e-02 -1.77597790e-03
 -5.98636940e-02 -9.78483260e-02  4.11238112e-02  6.72471374e-02
  2.91385315e-02  9.27953701e-03 -6.43227696e-02  1.65153965e-02
 -2.09651273e-02 -3.25862691e-02  4.39033005e-03 -4.31389697e-02
 -4.13816646e-02  5.29966094e-02  1.04158167e-02 -2.10031960e-02
 -2.20258925e-02  4.15516011e-02 -1.92304607e-02  6.8459

encoder.encoders.7.norm1.bias 	 [ 9.94000375e-01 -2.16676318e-03  5.81716359e-01  1.15108407e+00
  7.99299717e-01 -1.34397283e-01 -7.72202671e-01  3.79985958e-01
 -8.27040002e-02 -4.23146129e-01 -5.34657873e-02 -1.46756440e-01
 -6.53315961e-01  5.59262991e-01  1.58611000e-01  2.11146563e-01
  5.07956088e-01 -5.93862496e-02  1.76201746e-01 -6.97211683e-01
  5.14406443e-01 -4.22573000e-01 -4.28431302e-01 -1.83819249e-01
 -7.33476639e-01 -6.18022323e-01  2.87674600e-04  1.93128765e-01
  1.95516616e-01  5.28779961e-02  4.40624207e-01  2.16774613e-01
  5.42841733e-01 -1.65945351e-01  7.84309953e-02  3.22896957e-01
  6.41552746e-01  8.08654249e-01  6.87528312e-01  1.58587828e-01
 -6.72208011e-01  7.17587113e-01  7.56677449e-01 -3.21384579e-01
  3.88281167e-01  2.29152162e-02  6.42715216e-01 -1.18648127e-01
  9.00118947e-02 -2.42056549e-01  5.89506090e-01  1.93561539e-01
 -2.46124998e-01 -6.44177794e-01 -2.27652341e-01 -7.69496441e-01
  7.42253542e-01  5.19911289e-01  2.86197931e-01 -1.425495

AttributeError: 'NoneType' object has no attribute 'detach'

In [60]:
model, train_args = load_trained_model(ckpt)
model.eval()

x = np.expand_dims(feat, 0)
ilen = np.array(feat.shape[0]).reshape(1)
y = np.expand_dims(label2, 0)
print(x.shape)
print(ilen)
print(ilen.shape)
print(y.shape)
print(y)

device='cpu'
model.to(device)
model.zero_grad()
print(model.adim)

xs_pad = torch.as_tensor(x).to(device=device)
ilens = torch.as_tensor(ilen).to(device)
ys_pad = torch.as_tensor(y).to(device)
    

xs_pad = xs_pad[:, : max(ilens)]
src_mask = make_non_pad_mask(ilens.tolist()).to(xs_pad.device).unsqueeze(-2)
hs_pad, hs_mask = model.encoder(xs_pad, src_mask)
print(hs_pad.sum())
batch_size = xs_pad.size(0)
hs_len = hs_mask.view(batch_size, -1).sum(1)
loss = model.ctc(hs_pad.view(batch_size, -1, model.adim), hs_len, ys_pad)

print('loss', loss)
loss.retain_grad()
loss.backward()
print(loss.grad)

(1, 584, 83)
[584]
(1,)
(1, 14)
[[3152 2598 3008 3630  245  410    2  389 2893 2770 2572 4499 2391 2926]]
512
tensor(122.5361, grad_fn=<SumBackward0>)
loss tensor(305.5013, grad_fn=<SumBackward0>)
tensor(1.)


In [61]:
# with np.printoptions(threshold=np.inf):
for key, val in model.named_parameters():
    if 'decoder' in key: 
        continue
    print(key, "\t",  val.grad.detach().numpy())

encoder.embed.conv.0.weight 	 [[[[  0.         0.         0.      ]
   [  0.         0.         0.      ]
   [  0.         0.         0.      ]]]


 [[[-35.328438 -32.877594 -40.162292]
   [-41.746864 -33.297127 -45.857784]
   [-39.91947  -34.858356 -36.297985]]]


 [[[  0.         0.         0.      ]
   [  0.         0.         0.      ]
   [  0.         0.         0.      ]]]


 ...


 [[[  0.         0.         0.      ]
   [  0.         0.         0.      ]
   [  0.         0.         0.      ]]]


 [[[  0.         0.         0.      ]
   [  0.         0.         0.      ]
   [  0.         0.         0.      ]]]


 [[[  0.         0.         0.      ]
   [  0.         0.         0.      ]
   [  0.         0.         0.      ]]]]
encoder.embed.conv.0.bias 	 [  0.         -32.353756     0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.         -32.968864     0.           0.
  46.340008     0.           0.           0.          

encoder.encoders.4.norm2.bias 	 [ 4.95346606e-01 -1.86014906e-01  1.49989522e+00 -5.42997479e-01
  4.79654402e-01  1.68489948e-01  1.59401071e+00  1.42462790e-01
 -3.67995411e-01 -1.65345919e+00  1.48489761e+00 -6.34604692e-01
  3.29183817e+00 -1.75928223e+00  1.11529493e+00  2.77781248e+00
 -4.83611166e-01  3.09353080e-02  3.55925113e-01 -4.56506848e-01
  1.72594225e+00 -5.13319910e-01  1.69244456e+00 -2.35638452e+00
 -5.93583807e-02 -3.11441362e-01  6.26785517e-01  8.58471334e-01
 -2.06104636e+00  1.49670017e+00  2.54052496e+00 -2.36507368e+00
  2.65974045e-01 -1.14294875e+00 -1.70338243e-01 -1.67508602e+00
 -1.12496245e+00  1.79864812e+00  1.81441045e+00 -4.66356248e-01
  2.83094078e-01  2.21143484e+00  1.50627863e+00  2.79052687e+00
 -9.33230639e-01 -1.18583187e-01 -3.75719398e-01  1.23431969e+00
  3.27370930e+00  1.11861491e+00 -6.19915128e-01 -1.68303514e+00
 -3.18848968e+00  2.86668229e+00  1.34767532e-01 -2.51868296e+00
 -2.88024008e-01 -2.40591741e+00 -2.16066408e+00 -8.446476

encoder.encoders.10.self_attn.linear_v.weight 	 [[ 0.00333382 -0.00299478 -0.0132814  ...  0.01544517  0.00501803
   0.0094731 ]
 [ 0.00451357 -0.0053751   0.00330726 ...  0.00173223 -0.00201114
  -0.0046627 ]
 [ 0.00033703  0.00053943 -0.00487974 ...  0.005888   -0.00417736
   0.00140359]
 ...
 [-0.00735803  0.01637845  0.00113665 ...  0.0117127  -0.00351232
  -0.00952099]
 [-0.00939481 -0.00204567  0.01037352 ... -0.01145862 -0.00051575
   0.00448795]
 [-0.00230364 -0.00721369 -0.00582005 ... -0.00494031  0.00513084
  -0.01095319]]
encoder.encoders.10.self_attn.linear_v.bias 	 [ 7.62150437e-02  3.25861722e-02  5.25492951e-02 -9.10833627e-02
 -1.04891576e-01  2.97617555e-01  1.10186130e-01  5.06273500e-05
 -7.40201175e-02  8.19385350e-02  5.41624017e-02 -1.58273593e-01
  1.44895082e-02 -1.60119981e-01 -1.80281758e-01 -9.24946591e-02
  8.55683982e-02 -1.31974434e-02 -1.50026828e-01  1.30096793e-01
  6.60416335e-02 -6.83845878e-02 -9.76766273e-02 -8.54832157e-02
 -3.78591418e-02  5.7921

In [32]:
# paddle transformer

In [33]:
!ls ../DeepSpeech-2.x/examples/librispeech/s2/conf/transformer.yaml

../DeepSpeech-2.x/examples/librispeech/s2/conf/transformer.yaml


In [34]:
config_path = '../DeepSpeech-2.x/examples/librispeech/s2/conf/transformer.yaml'

In [ ]:
!pip install yacs

In [35]:
from yacs.config import CfgNode

In [36]:
config = CfgNode()
config.set_new_allowed(True)
config.merge_from_file(config_path)

In [37]:
config = CfgNode.load_cfg(open(config_path, 'rt'))
print(config)

collator:
  augmentation_config: conf/augmentation.json
  batch_bins: 0
  batch_count: auto
  batch_frames_in: 0
  batch_frames_inout: 0
  batch_frames_out: 0
  batch_size: 32
  feat_dim: 83
  maxlen_in: 512
  maxlen_out: 150
  minibatches: 0
  num_encs: 1
  num_workers: 2
  sortagrad: 0
  spm_model_prefix: data/train_960_unigram5000
  stride_ms: 10.0
  subsampling_factor: 1
  unit_type: spm
  vocab_filepath: data/train_960_unigram5000_units.txt
  window_ms: 25.0
data:
  dev_manifest: data/manifest.dev
  test_manifest: data/manifest.test-clean
  train_manifest: data/manifest.train
decoding:
  alpha: 2.5
  batch_size: 64
  beam_size: 10
  beta: 0.3
  ctc_weight: 0.5
  cutoff_prob: 1.0
  cutoff_top_n: 0
  decoding_chunk_size: -1
  decoding_method: attention
  error_rate_type: wer
  lang_model_path: data/lm/common_crawl_00.prune01111.trie.klm
  num_decoding_left_chunks: -1
  num_proc_bsearch: 8
  simulate_streaming: False
model:
  cmvn_file: None
  cmvn_file_type: json
  decoder: transfor

In [210]:
from deepspeech.models.u2 import U2Model
import paddle

In [39]:
config.model.input_dim = 83
config.model.output_dim = 5002
config.model.encoder_conf.output_size = 512
pmodel = U2Model.from_config(config.model)

[INFO 2021/08/30 10:47:20 u2.py:854] U2 Encoder type: transformer


In [40]:
print(pmodel)

U2Model(
  (encoder): TransformerEncoder(
    (embed): Conv2dSubsampling4(
      (pos_enc): PositionalEncoding(
        (dropout): Dropout(p=0.1, axis=None, mode=upscale_in_train)
      )
      (conv): Sequential(
        (0): Conv2D(1, 512, kernel_size=[3, 3], stride=[2, 2], data_format=NCHW)
        (1): ReLU()
        (2): Conv2D(512, 512, kernel_size=[3, 3], stride=[2, 2], data_format=NCHW)
        (3): ReLU()
      )
      (out): Sequential(
        (0): Linear(in_features=10240, out_features=512, dtype=float32)
      )
    )
    (after_norm): LayerNorm(normalized_shape=[512], epsilon=1e-12)
    (encoders): LayerList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linear_q): Linear(in_features=512, out_features=512, dtype=float32)
          (linear_k): Linear(in_features=512, out_features=512, dtype=float32)
          (linear_v): Linear(in_features=512, out_features=512, dtype=float32)
          (linear_out): Linear(in_features=512, out_f

In [41]:
for key, val in pmodel.named_parameters():
    print(key, '\t', val.shape)

encoder.embed.conv.0.weight 	 [512, 1, 3, 3]
encoder.embed.conv.0.bias 	 [512]
encoder.embed.conv.2.weight 	 [512, 512, 3, 3]
encoder.embed.conv.2.bias 	 [512]
encoder.embed.out.0.weight 	 [10240, 512]
encoder.embed.out.0.bias 	 [512]
encoder.after_norm.weight 	 [512]
encoder.after_norm.bias 	 [512]
encoder.encoders.0.self_attn.linear_q.weight 	 [512, 512]
encoder.encoders.0.self_attn.linear_q.bias 	 [512]
encoder.encoders.0.self_attn.linear_k.weight 	 [512, 512]
encoder.encoders.0.self_attn.linear_k.bias 	 [512]
encoder.encoders.0.self_attn.linear_v.weight 	 [512, 512]
encoder.encoders.0.self_attn.linear_v.bias 	 [512]
encoder.encoders.0.self_attn.linear_out.weight 	 [512, 512]
encoder.encoders.0.self_attn.linear_out.bias 	 [512]
encoder.encoders.0.feed_forward.w_1.weight 	 [512, 2048]
encoder.encoders.0.feed_forward.w_1.bias 	 [2048]
encoder.encoders.0.feed_forward.w_2.weight 	 [2048, 512]
encoder.encoders.0.feed_forward.w_2.bias 	 [512]
encoder.encoders.0.norm1.weight 	 [512]
encode

In [42]:
for key, val in paddle_model_state_dict.items():
    print(key, '\t', val.shape)

encoder.embed.conv.0.weight 	 (512, 1, 3, 3)
encoder.embed.conv.0.bias 	 (512,)
encoder.embed.conv.2.weight 	 (512, 512, 3, 3)
encoder.embed.conv.2.bias 	 (512,)
encoder.embed.out.0.weight 	 (10240, 512)
encoder.embed.out.0.bias 	 (512,)
encoder.encoders.0.self_attn.linear_q.weight 	 (512, 512)
encoder.encoders.0.self_attn.linear_q.bias 	 (512,)
encoder.encoders.0.self_attn.linear_k.weight 	 (512, 512)
encoder.encoders.0.self_attn.linear_k.bias 	 (512,)
encoder.encoders.0.self_attn.linear_v.weight 	 (512, 512)
encoder.encoders.0.self_attn.linear_v.bias 	 (512,)
encoder.encoders.0.self_attn.linear_out.weight 	 (512, 512)
encoder.encoders.0.self_attn.linear_out.bias 	 (512,)
encoder.encoders.0.feed_forward.w_1.weight 	 (512, 2048)
encoder.encoders.0.feed_forward.w_1.bias 	 (2048,)
encoder.encoders.0.feed_forward.w_2.weight 	 (2048, 512)
encoder.encoders.0.feed_forward.w_2.bias 	 (512,)
encoder.encoders.0.norm1.weight 	 (512,)
encoder.encoders.0.norm1.bias 	 (512,)
encoder.encoders.0.norm

In [162]:
pmodel.set_state_dict(paddle_model_state_dict)

In [163]:
for key, val in pmodel.state_dict().items():
    if 'concat_linear' in key:
        continue
    print(key)
    print(np.allclose(val, paddle_model_state_dict[key]))

encoder.embed.conv.0.weight
True
encoder.embed.conv.0.bias
True
encoder.embed.conv.2.weight
True
encoder.embed.conv.2.bias
True
encoder.embed.out.0.weight
True
encoder.embed.out.0.bias
True
encoder.after_norm.weight
True
encoder.after_norm.bias
True
encoder.encoders.0.self_attn.linear_q.weight
True
encoder.encoders.0.self_attn.linear_q.bias
True
encoder.encoders.0.self_attn.linear_k.weight
True
encoder.encoders.0.self_attn.linear_k.bias
True
encoder.encoders.0.self_attn.linear_v.weight
True
encoder.encoders.0.self_attn.linear_v.bias
True
encoder.encoders.0.self_attn.linear_out.weight
True
encoder.encoders.0.self_attn.linear_out.bias
True
encoder.encoders.0.feed_forward.w_1.weight
True
encoder.encoders.0.feed_forward.w_1.bias
True
encoder.encoders.0.feed_forward.w_2.weight
True
encoder.encoders.0.feed_forward.w_2.bias
True
encoder.encoders.0.norm1.weight
True
encoder.encoders.0.norm1.bias
True
encoder.encoders.0.norm2.weight
True
encoder.encoders.0.norm2.bias
True
encoder.encoders.1.sel

True
decoder.output_layer.bias
True
decoder.decoders.0.self_attn.linear_q.weight
True
decoder.decoders.0.self_attn.linear_q.bias
True
decoder.decoders.0.self_attn.linear_k.weight
True
decoder.decoders.0.self_attn.linear_k.bias
True
decoder.decoders.0.self_attn.linear_v.weight
True
decoder.decoders.0.self_attn.linear_v.bias
True
decoder.decoders.0.self_attn.linear_out.weight
True
decoder.decoders.0.self_attn.linear_out.bias
True
decoder.decoders.0.src_attn.linear_q.weight
True
decoder.decoders.0.src_attn.linear_q.bias
True
decoder.decoders.0.src_attn.linear_k.weight
True
decoder.decoders.0.src_attn.linear_k.bias
True
decoder.decoders.0.src_attn.linear_v.weight
True
decoder.decoders.0.src_attn.linear_v.bias
True
decoder.decoders.0.src_attn.linear_out.weight
True
decoder.decoders.0.src_attn.linear_out.bias
True
decoder.decoders.0.feed_forward.w_1.weight
True
decoder.decoders.0.feed_forward.w_1.bias
True
decoder.decoders.0.feed_forward.w_2.weight
True
decoder.decoders.0.feed_forward.w_2.bi

In [257]:
# paddle encoder forward 

In [258]:
paddle.set_device('cpu')
pmodel.eval()

x = np.expand_dims(feat, 0)
ilen = np.array(feat.shape[0]).reshape(1)
y = np.expand_dims(label, 0)
olen = np.array(len(label)).reshape(1)
print(x.shape)
print(ilen)
print(ilen.shape)
print(y.shape)
print(olen.shape)
print(olen)
print(y)

pmodel.clear_gradients()

p_xs_pad = paddle.to_tensor(x)
p_ilens = paddle.to_tensor(ilen)
p_text = paddle.to_tensor(y)
p_text_lengths = paddle.to_tensor(olen)


encoder_out, encoder_mask = pmodel.encoder(p_xs_pad, p_ilens)
print(encoder_out.shape, encoder_mask.shape)
print(encoder_out.sum())
encoder_out_lens = encoder_mask.squeeze(1).cast(paddle.int64).sum(1)  #[B, 1, T] -> [B]
print(encoder_out_lens)

loss_ctc = pmodel.ctc(encoder_out, encoder_out_lens, p_text,
                                p_text_lengths)
print('loss', loss_ctc.numpy())
loss_ctc.backward(retain_graph=True)
print(loss_ctc.grad.numpy())

(1, 584, 83)
[584]
(1,)
(1, 22)
(1,)
[22]
[[3008 3630  245  410 2598 4502  482  352  399  252 3204 4502 2974 1165
   627 4845  689 2209 4577 4859 2391 2237]]
[1, 145, 512] [1, 1, 145]
Tensor(shape=[1], dtype=float32, place=CPUPlace, stop_gradient=False,
       [84.23927307])
Tensor(shape=[1], dtype=int64, place=CPUPlace, stop_gradient=True,
       [145])
loss [63.926617]
[1.]


In [259]:

from espnet.utils.dynamic_import import dynamic_import
def load_trained_model(model_path, training=True):
    """Load the trained model for recognition.

    Args:
        model_path (str): Path to model.***.best

    """
    idim, odim, train_args = get_model_conf(
        model_path, os.path.join(os.path.dirname(model_path), "model.json")
    )

    logging.warning("reading model parameters from " + model_path)

    if hasattr(train_args, "model_module"):
        model_module = train_args.model_module
    else:
        model_module = "espnet.nets.pytorch_backend.e2e_asr:E2E"
    # CTC Loss is not needed, default to builtin to prevent import errors
#     if hasattr(train_args, "ctc_type"):
#         train_args.ctc_type = "builtin"

    model_class = dynamic_import(model_module)

    if "transducer" in model_module:
        model = model_class(idim, odim, train_args, training=training)
        custom_torch_load(model_path, model, training=training)
    else:
        model = model_class(idim, odim, train_args)
        torch_load(model_path, model)

    return model, train_args

In [260]:
model, train_args = load_trained_model(ckpt)

In [261]:
import warpctc_pytorch as warp_ctc

ctc = warp_ctc.CTCLoss(size_average=True, reduce=True)

In [262]:
!cat ../asr/espnet//egs/librispeech/asr1/asr1/exp/train_960_pytorch_train_specaug/results/model.json | grep ctc

grep: warning: GREP_OPTIONS is deprecated; please use an alias or script
        "ctc_type": "warpctc",
        "ctc_weight": 0.3,
        "num_save_ctc": 3,


In [263]:
print(train_args.ctc_type)

warpctc


In [264]:
import torch.nn.functional as F
def to_device(m, x):
    """Send tensor into the device of the module.

    Args:
        m (torch.nn.Module): Torch module.
        x (Tensor): Torch tensor.

    Returns:
        Tensor: Torch tensor located in the same place as torch module.

    """
    if isinstance(m, torch.nn.Module):
        device = next(m.parameters()).device
    elif isinstance(m, torch.Tensor):
        device = m.device
    else:
        raise TypeError(
            "Expected torch.nn.Module or torch.tensor, " f"bot got: {type(m)}"
        )
    return x.to(device)


In [265]:
model.load_state_dict(model_state_dict)
model.eval()

x = np.expand_dims(feat, 0)
ilen = np.array(feat.shape[0]).reshape(1)
y = np.expand_dims(label, 0)
print(x.shape)
print(ilen)
print(ilen.shape)
print(y.shape)
print(y)

device='cpu'
model.to(device)
model.zero_grad()
print(model.adim)

xs_pad = torch.as_tensor(x).to(device=device)
ilens = torch.as_tensor(ilen).to(device)
ys_pad = torch.as_tensor(y).to(device)
    

xs_pad = xs_pad[:, : max(ilens)]
src_mask = make_non_pad_mask(ilens.tolist()).to(xs_pad.device).unsqueeze(-2)
hs_pad, hs_mask = model.encoder(xs_pad, src_mask)
print(hs_pad.sum())


batch_size = xs_pad.size(0)
hlens = hs_mask.view(batch_size, -1).sum(1)
ys = [y[y != model.ctc.ignore_id] for y in ys_pad]  # parse padded ys
ys_hat = model.ctc.ctc_lo(F.dropout(hs_pad, p=0.0))
ys_hat = ys_hat.transpose(0, 1)
olens = to_device(ys_hat, torch.LongTensor([len(s) for s in ys]))
hlens = hlens.long()
ys_pad = torch.cat(ys)  # without this the code breaks for asr_mix
loss = model.ctc.loss_fn(ys_hat, ys_pad, hlens, olens)

print('loss', loss)
loss.retain_grad()
loss.backward()
print(loss.grad)

(1, 584, 83)
[584]
(1,)
(1, 22)
[[3008 3630  245  410 2598 4502  482  352  399  252 3204 4502 2974 1165
   627 4845  689 2209 4577 4859 2391 2237]]
512
tensor(122.5360, grad_fn=<SumBackward0>)
loss tensor([169.9004], grad_fn=<_CTCBackward>)
tensor([1.])


In [266]:

def p_make_pad_mask(lengths: paddle.Tensor) -> paddle.Tensor:
    """Make mask tensor containing indices of padded part.
    See description of make_non_pad_mask.
    Args:
        lengths (paddle.Tensor): Batch of lengths (B,).
    Returns:
        paddle.Tensor: Mask tensor containing indices of padded part.
    Examples:
        >>> lengths = [5, 3, 2]
        >>> make_pad_mask(lengths)
        masks = [[0, 0, 0, 0 ,0],
                 [0, 0, 0, 1, 1],
                 [0, 0, 1, 1, 1]]
    """
    # (TODO: Hui Zhang): jit not support Tenosr.dim() and Tensor.ndim
    # assert lengths.dim() == 1
    batch_size = int(lengths.shape[0])
    max_len = int(lengths.max())
    seq_range = paddle.arange(0, max_len, dtype=paddle.int64)
    seq_range_expand = seq_range.unsqueeze(0).expand([batch_size, max_len])
    seq_length_expand = lengths.unsqueeze(-1)
    mask = seq_range_expand >= seq_length_expand
    return mask


def p_make_non_pad_mask(lengths: paddle.Tensor) -> paddle.Tensor:
    """Make mask tensor containing indices of non-padded part.
    The sequences in a batch may have different lengths. To enable
    batch computing, padding is need to make all sequence in same
    size. To avoid the padding part pass value to context dependent
    block such as attention or convolution , this padding part is
    masked.
    This pad_mask is used in both encoder and decoder.
    1 for non-padded part and 0 for padded part.
    Args:
        lengths (paddle.Tensor): Batch of lengths (B,).
    Returns:
        paddle.Tensor: mask tensor containing indices of padded part.
    Examples:
        >>> lengths = [5, 3, 2]
        >>> make_non_pad_mask(lengths)
        masks = [[1, 1, 1, 1 ,1],
                 [1, 1, 1, 0, 0],
                 [1, 1, 0, 0, 0]]
    """
    #TODO(Hui Zhang): return ~make_pad_mask(lengths), not support ~
    return p_make_pad_mask(lengths).logical_not()

In [281]:

# p_xs_pad = paddle.to_tensor(x)
# p_ilens = paddle.to_tensor(ilen)
# p_text = paddle.to_tensor(y)
# p_text_lengths = paddle.to_tensor(olen)

# encoder_out, encoder_mask = pmodel.encoder(p_xs_pad, p_ilens)
# print(encoder_out.shape, encoder_mask.shape)
# print(encoder_out.sum())
# encoder_out_lens = encoder_mask.squeeze(1).cast(paddle.int64).sum(1)  #[B, 1, T] -> [B]
# print(encoder_out_lens)

# loss_ctc = pmodel.ctc(encoder_out, encoder_out_lens, p_text,
#                                 p_text_lengths)


print('# input')
print(np.allclose(p_xs_pad, xs_pad))
print(np.allclose(p_ilens, ilens))
print(np.allclose(p_text, ys_pad))
print(np.allclose(p_text_lengths, olens))

print('# encoder output')
print(np.allclose(encoder_mask, hs_mask))
print(np.allclose(encoder_out.numpy(), hs_pad.detach().numpy()))

print('# encoder embed')
masks = p_make_non_pad_mask(p_ilens).unsqueeze(1)  # (B, 1, L)
pxs, pos_emb, pmasks,  = pmodel.encoder.embed(p_xs_pad, masks)
xs, masks = model.encoder.embed(xs_pad, src_mask)
print(np.allclose(pxs.numpy(), xs.detach().numpy(), atol=1e-3, rtol=0))
print(np.allclose(pmasks.numpy(), masks.detach().numpy()))


from deepspeech.modules.mask import add_optional_chunk_mask
print('# encoder encoders')
pxs = paddle.to_tensor(xs.detach().numpy())
pmasks = paddle.to_tensor(masks.detach().numpy())
pmasks = pmasks.astype(paddle.bool)
mask_pad = ~pmasks
chunk_masks = add_optional_chunk_mask(
    pxs, pmasks, False, False,
    0, 0,
    0)
print(type(chunk_masks))
for layer in pmodel.encoder.encoders:
    pxs, chunk_masks, _ = layer.self_attn(pxs, chunk_masks, pos_emb, mask_pad)
    layer0 = layer
    break

    
# xs, masks = model.encoder.encoders(xs, masks)
for layer in model.encoder.encoders:
    xs, masks = layer.self_attn(xs, masks)
    tlayer0 = layer
    break;
print(np.allclose(chunk_masks.numpy(), masks.detach().numpy(), atol=1e-3))
print(np.allclose(pxs.numpy(), xs.detach().numpy(), atol=1e-1))
print(pxs.numpy())
print(xs.detach().numpy())

# input
True
True
True
True
# encoder output
True
False
# encoder embed
True
True
# encoder encoders
<class 'paddle.Tensor'>


RuntimeError: (NotFound) Operator matmul does not have kernel for data_type[bool]:data_layout[ANY_LAYOUT]:place[CPUPlace]:library_type[PLAIN].
  [Hint: Expected kernel_iter != kernels.end(), but received kernel_iter == kernels.end().] (at /paddle/paddle/fluid/imperative/prepared_operator.cc:159)
  [operator < matmul > error]

In [280]:
print(layer0.self_attn.linear_q.weight)
print(tlayer0.self_attn.linear_q.weight)

Parameter containing:
Tensor(shape=[512, 512], dtype=float32, place=CPUPlace, stop_gradient=False,
       [[ 0.13450158, -0.82629627, -0.72592098, ...,  0.17605539,  0.78236377, -0.07586893],
        [ 0.03805150, -0.71657342,  0.67314267, ..., -0.06278662,  0.30976444, -0.34616092],
        [ 0.56945264, -0.37384635, -0.02534366, ...,  0.27068651,  0.05874859, -0.49176145],
        ...,
        [-0.97151852, -0.23421702,  0.26694256, ...,  0.01556672, -0.22618203, -0.30741346],
        [-0.51141077, -1.06890213, -0.18566237, ..., -0.24530560,  0.04875533,  0.20320544],
        [-0.51901811,  0.64869654,  0.15846817, ..., -0.92547667, -0.17422923,  0.07760677]])
Parameter containing:
tensor([[ 0.1345,  0.0381,  0.5695,  ..., -0.9715, -0.5114, -0.5190],
        [-0.8263, -0.7166, -0.3738,  ..., -0.2342, -1.0689,  0.6487],
        [-0.7259,  0.6731, -0.0253,  ...,  0.2669, -0.1857,  0.1585],
        ...,
        [ 0.1761, -0.0628,  0.2707,  ...,  0.0156, -0.2453, -0.9255],
        [ 0.78